<a href="https://colab.research.google.com/github/p09323028/2020f_NTU_Econometrics_I/blob/main/Textbook/CH6_Linear_Regression_with_Multiple_Regressors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 6: Linear Regression with Multiple Regressors**
Author: Jinze Wu

Student Number: p09323028

前置作業:
- import 套件
- 載入資料
- 讀取資料

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind

In [ ]:
!gdown --id '10Q4-t5L8KZjaxUqR8iGnmXx_lY9JYgSL' --output caschool.xlsx

In [ ]:
caschool = pd.read_excel('caschool.xlsx')

**Decription of Data;**

dist_code -- district Code;

Read_scr  -- avg Reading Score;

Math_scr  -- avg Math Score;

County   --  county;

District -- District;

gr_span -- grade span of district;

enrl_tot -- total enrollment;

teachers -- number of teachers;

computer -- number of computers;

testscr -- avg test score (= (read_scr+math_scr)/2 );

comp_stu -- computers per student ( = computer/enrl_tot);

expn_stu -- expentitures per student;

str -- student teacher ration (teachers/enrl_tot);

el_pct -- percent of English Learners;

Meal_pct -- Percent qualifying for reduced-price lunch;

clw_pct -- Percent qualifying for CalWorks;

aving -- district average income (in $1000's);

## **6.1 Omitted Variable Bias**

### TABLE 6.1

In [ ]:
caschool['str_20'] = caschool.str.apply(lambda x: 1 if x < 20 else 0)
caschool['elq1'] = caschool.el_pct.apply(lambda x: 1 if x < 1.9 else 0)
caschool['elq2'] = caschool.el_pct.apply(lambda x: 1 if (x >= 1.9 and x < 8.8) else 0)
caschool['elq3'] = caschool.el_pct.apply(lambda x: 1 if (x >= 8.8 and x < 23.0) else 0)
caschool['elq4'] = caschool.el_pct.apply(lambda x: 1 if x > 23.0 else 0)

In [ ]:
ts_lostr = caschool[caschool['str_20']==1]
ts_histr = caschool[caschool['str_20']==0]

#### All districts

In [ ]:
results = ttest_ind(x1=ts_lostr.testscr, x2=ts_histr.testscr, usevar='unequal')
print('t={:.4f}'.format(results[0]))
print('pvalue={:.4f}'.format(results[1]))
print('df={:.4f}'.format(results[2]))

t=4.0426
pvalue=0.0001
df=403.6070


####  $<1.9\%$

In [ ]:
results = ttest_ind(x1=ts_lostr[ts_lostr['elq1']==1].testscr, x2=ts_histr[ts_histr['elq1']==1].testscr, usevar='unequal')
print('t={:.4f}'.format(results[0]))
print('pvalue={:.4f}'.format(results[1]))
print('df={:.4f}'.format(results[2]))

t=-0.3039
pvalue=0.7623
df=59.4242


####  $1.9-8.8\%$

In [ ]:
results = ttest_ind(x1=ts_lostr[ts_lostr['elq2']==1].testscr, x2=ts_histr[ts_histr['elq2']==1].testscr, usevar='unequal')
print('t={:.4f}'.format(results[0]))
print('pvalue={:.4f}'.format(results[1]))
print('df={:.4f}'.format(results[2]))

t=1.1392
pvalue=0.2572
df=105.3476


####  $8.8-23.3\%$

In [ ]:
results = ttest_ind(x1=ts_lostr[ts_lostr['elq3']==1].testscr, x2=ts_histr[ts_histr['elq3']==1].testscr, usevar='unequal')
print('t={:.4f}'.format(results[0]))
print('pvalue={:.4f}'.format(results[1]))
print('df={:.4f}'.format(results[2]))

t=1.7208
pvalue=0.0884
df=99.1758


####  $>23.3\%$

In [ ]:
results = ttest_ind(x1=ts_lostr[ts_lostr['elq4']==1].testscr, x2=ts_histr[ts_histr['elq4']==1].testscr, usevar='unequal')
print('t={:.4f}'.format(results[0]))
print('pvalue={:.4f}'.format(results[1]))
print('df={:.4f}'.format(results[2]))

t=0.6786
pvalue=0.4994
df=79.0952


## **6.2 The Multiple Regression Model**

## **6.3 The OLS Estimator in Multiple Regression**

### Equation 6.11

$\widehat{TestScore} = 698.9 - 2.28 \times STR$

In [ ]:
reg_6_11 = smf.ols(formula='testscr~str', data=caschool)
results_6_11 = reg_6_11.fit(cov_type='HC1')  # robust
print(results_6_11.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     19.26
Date:                Thu, 08 Apr 2021   Prob (F-statistic):           1.45e-05
Time:                        09:05:09   Log-Likelihood:                -1822.2
No. Observations:                 420   AIC:                             3648.
Df Residuals:                     418   BIC:                             3657.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    698.9330     10.364     67.436      0.0

### Equation 6.11

$\widehat{TestScore} = 686.0 - 1.10 \times STR - 0.65 \times PctEL$

In [ ]:
reg_6_12 = smf.ols(formula='testscr~str+el_pct', data=caschool)
results_6_12 = reg_6_12.fit(cov_type='HC1')  # robust
print(results_6_12.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     223.8
Date:                Thu, 08 Apr 2021   Prob (F-statistic):           9.28e-67
Time:                        09:06:36   Log-Likelihood:                -1716.6
No. Observations:                 420   AIC:                             3439.
Df Residuals:                     417   BIC:                             3451.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    686.0322      8.728     78.599      0.0

## **6.4 Measures of Fit in Multiple Regression**